<h1> Shamir's Secret Sharing </h1>
k of n threshold scheme. <br>
This particular implementation is to encode a plaintext string to hex shares, which can later be recombined.<br>
The length of the plaintext secret string that can be split and shared using this scheme is limited by the size of the prime number used.

Importing required libraries

In [2]:
import random 
import math 

Defining the threshold k, the number of shares n and the prime number required for the modular math

In [3]:
random.seed(1001)
secretString="bandrobot band had malfunctioned"
threshold=2
numberOfShares=6
degree=threshold-1
prime=115792089237316195423570985008687907853269984665640564039457584007913129639747

Function to encode plaintext string to hex

In [4]:
def encodeToHex(st):
    temp=st.encode("utf-8").hex()
    return "0x"+temp

Function to decode hex string to plaintext

In [5]:
def decodeFromHex(st):
    #print("here",st,"\n")
    y=bytearray.fromhex(st).decode()
    return y

Splitting of secret into shares.

In [6]:
def splitShares(secretString):
    if(threshold>numberOfShares):
        raise ValueError("Threshold must be lower than number of shares")
    if(threshold<2):
        raise ValueError("Threshold must at least be 2")
    temp=encodeToHex(secretString)    
    secretInt = int(temp,16)    
    points = secretIntToPoints(secretInt)
    shares = []
    for point in points:
        shares.append(pointToShareString(point))
    return shares

Generation of random polynomial of which the free coefficient is the secret.

In [7]:
def randomPolynomial(degree,secretInt,prime): 
    coefficients=[]
    coefficients.append(secretInt) #a0 is the secret itself
    for i in range(degree):
        random_coeff = random.randint(0, prime-1) #other coeffs are randomly generated
        coefficients.append(random_coeff)
    return coefficients

From the randomly generated polynomial, pick n points each of which corresponds to a share of the secret.

In [8]:
def getPolynomialPoints(coefficients,numberOfShares,prime):
    points=[]
    for x in range(1, numberOfShares+1):
        y = coefficients[0]
        # calculate each term and add it to y, using modular math
        for i in range(1, len(coefficients)):
            exponentiation = (x**i) % prime
            term = (coefficients[i] * exponentiation) % prime
            y = (y + term) % prime
        # add the point to the list of points
        points.append((x, y))
    return points

Function to generate a random polynomial from the secret integer, and then generate points from that polynomial.

In [9]:
def secretIntToPoints(secretInt):
    coefficients = randomPolynomial(degree,secretInt,prime)    
    points = getPolynomialPoints(coefficients, numberOfShares,prime)    
    return points

Generate a hex string corresponding to each point from the polynomial. Each share is of the form i-f(i)

In [10]:
def pointToShareString(point):
    x,y=point
    return hex(x)+"-"+hex(y)

Generate the corresponding point given the hex share string

In [11]:
def shareStringToPoint(shareString):
    x_str,y_str=shareString.split('-')
    x=int(x_str,16)
    y=int(y_str,16)
    return (x,y)

Retrieve the secret integer from the points generated by interpolating the polynomial using Modulr Lagrange interpolation.

In [12]:
def pointsToSecretInt(points):
    freeCoefficient = modularLagrangeInterpolation(0, points, prime)
    secretInt = freeCoefficient  # the secret int is the free coefficient
    return secretInt

Using extended Euclid's algorithm to find GCD

In [13]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

Computing a modular inverse given a finite field of a prime number P.

In [14]:
def mod_inverse(k, prime):
    k = k % prime
    if k < 0:
        r = egcd(prime, -k)[2]
    else:
        r = egcd(prime, k)[2]
    return (prime + r) % prime

Lagrange interpolation works by finding the Lagrange basis polynomial for every point in the set. A linear combination of the basis polynomials scaled by the y values at those points.

In [15]:
def modularLagrangeInterpolation(x, points, prime):    
    x_values, y_values = zip(*points)    
    f_x = 0
    for i in range(len(points)):        
        numerator=denominator = 1
        for j in range(len(points)):            
            if i == j:
                continue            
            numerator = (numerator * (x - x_values[j])) % prime
            denominator = (denominator * (x_values[i] - x_values[j])) % prime
        # get the polynomial from the numerator + denominator mod inverse
        lagrange_polynomial = numerator * mod_inverse(denominator, prime)
        # multiply the current y & the evaluated polynomial & add it to f(x)
        f_x = (prime + f_x + (y_values[i] * lagrange_polynomial)) % prime    
    return f_x

From knowledge of just the shares and the threshold, the secret is recovered by interpolation if a random polynomial.

In [16]:
def recoverSecret(a):
    print("Number of shares supplied:",len(a))
    print("Shares supplied:",a)
    print("Threshold required for secret recovery:",threshold)
    if(len(a)<threshold):
        raise ValueError("Not enough shares")

    points=[]
    for i in a:
        points.append(shareStringToPoint(i))
    #print("point2",points,"\n")
    sec=pointsToSecretInt(points) #after lagrange
    secHex=hex(sec)
    #print(sec,"\n")
    #print(secHex)
    print("\nRecovered secret:\n",decodeFromHex(str(secHex)[2:]))

Splitting shares and recovering them

In [17]:
a=splitShares(secretString)
print("All generated shares\n",a,"\n")
recoverSecret(a[6:])

All generated shares
 ['0x1-0x54375e634f49896aa6d2df0f31033364241d74db706e87695c1b663f3b5fb771', '0x2-0x460d4e622c23b065d9855bbcf3a2465fe6d6c9497f70a85d49d358150751097e', '0x3-0x37e33e6108fdd7610c37d86ab641595ba9901db78e72c951378b49ead3425b8b', '0x4-0x29b92e5fe5d7fe5c3eea551878e06c576c4972259d74ea4525433bc09f33ad98', '0x5-0x1b8f1e5ec2b22557719cd1c63b7f7f532f02c693ac770b3912fb2d966b24ffa5', '0x6-0xd650e5d9f8c4c52a44f4e73fe1e924ef1bc1b01bb792c2d00b31f6c371651b2'] 

Number of shares supplied: 0
Shares supplied: []
Threshold required for secret recovery: 2


ValueError: Not enough shares